In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
ash_excitation = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity


#Hamiltonian 
symbols = ["H","H","H","H"]
r_bohr = 3.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")

#print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

#Calculation of HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

#Putting all the functions here
#1.1091571486954503

Dr. Srivathsan changed it only the double excitation and not single
Singles are [FermiWord({(0, 0): '+', (1, 4): '-'}), FermiWord({(0, 0): '+', (1, 6): '-'}), FermiWord({(0, 1): '+', (1, 5): '-'}), FermiWord({(0, 1): '+', (1, 7): '-'}), FermiWord({(0, 2): '+', (1, 4): '-'}), FermiWord({(0, 2): '+', (1, 6): '-'}), FermiWord({(0, 3): '+', (1, 5): '-'}), FermiWord({(0, 3): '+', (1, 7): '-'})]
Doubles are [FermiWord({(0, 5): '+', (1, 4): '+', (2, 1): '-', (3, 0): '-'}), FermiWord({(0, 7): '+', (1, 4): '+', (2, 1): '-', (3, 0): '-'}), FermiWord({(0, 6): '+', (1, 5): '+', (2, 1): '-', (3, 0): '-'}), FermiWord({(0, 7): '+', (1, 6): '+', (2, 1): '-', (3, 0): '-'}), FermiWord({(0, 6): '+', (1, 4): '+', (2, 2): '-', (3, 0): '-'}), FermiWord({(0, 5): '+', (1, 4): '+', (2, 3): '-', (3, 0): '-'}), FermiWord({(0, 7): '+', (1, 4): '+', (2, 3): '-', (3, 0): '-'}), FermiWord({(0, 6): '+', (1, 5): '+', (2, 3): '-', (3, 0): '-'}), FermiWord({(0, 7): '+', (1, 6): '+', (2, 3): '-', (3, 0): '-'}), FermiWord

In [2]:
# Commutator calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_0(H,w, k):
    qml.BasisState(k, wires=range(qubits))
    res = qml.commutator(H, w)
    return qml.expval(res)

# Commutator calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_1(H,w, k):
    qml.StatePrep(k, wires=range(qubits))
    res = qml.commutator(H, w)
    return qml.expval(res)

# Energy calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def ash(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc going in is', ash_excitation[i])
            qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
            #qml.SingleExcitation(params[i], wires=ash_excitation[i])
    return qml.expval(H)

# Calculation of New state 
#print('Now we are going to calculate the new_state')
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def new_state(hf_state, ash_excitation, fparams, params):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('Exc. dealing right now is', ash_excitation[i])
            print('The params that are going in', params[i])
            print('The wires1 are', ash_excitation[i][2:][::-1])
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc. dealing right now is', ash_excitation[i])
            print('Single exc params that are going in', params[i])
            #qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
            qml.SingleExcitation(params[i], wires=ash_excitation[i])

    return qml.state()

In [3]:

electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

print('The Hamiltonian is ', H)
adit = 10
fparams = []
excitations= []
operator_pool = (singles) + (doubles)
states = [hf_state]


for j in range(adit):
    print('The adapt iteration now is', j)
    max_value = float('-inf')
    max_operator = None
    # Start with the most recent state (last state in the list)
    k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state

    for i in operator_pool:
        print('The current excitation operator is', i)
        w = qml.fermi.jordan_wigner(i)
        if np.array_equal(k, hf_state):
            print('Print, if this is activated - HF state')
            current_value = abs(commutator_0(H, w, k))
        else:
            current_value = abs(commutator_1(H, w, k))
        print(f'The expectation value of {i} is', current_value)

        if current_value > max_value:
            max_value = current_value
            max_operator = i

    print(f"The highest operator value is {max_value} for operator {max_operator}")

    # Convert operator to excitations and append to ash_excitation
    indices_str = re.findall(r'\d+', str(max_operator))
    excitations = [int(index) for index in indices_str]
    print('Highest gradient excitation is', excitations)
    ash_excitation.append(excitations)
    print('The current status of ash_excitation is', ash_excitation)
    print('Moving towards parameters')
    params = np.zeros(len(ash_excitation), requires_grad=True)
    print('The length of parameters is', len(params))
    
    # Cost function definition
    cost_fn = qml.QNode(ash, dev, interface="autograd", diff_method="adjoint")
    print('Going to do energy calculation')
    for n in range(150):
        print(f'Each step, the iteration is {n} and the parameter is {params}')
        params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits, H=H)
        if n % 5 == 0:
            print(f"step = {n}, E = {energy:.8f} Ha")
    fparams.append(params)
    print('Updated params are', params)
    print('Updated excitation are', ash_excitation)
    
    # New state generation
    ostate = new_state(hf_state, ash_excitation, fparams, params)
    print(qml.draw(new_state, level="device", max_length=100)(hf_state,ash_excitation,fparams,params))
    print('The updated state is', ostate)
    # Append the new state to the states list
    states.append(ostate)  


Dr. Srivathsan changed it only the double excitation and not single
The Hamiltonian is  -1.107840826225263 * I(0) + 0.050636412339760406 * Z(0) + 0.002022957331030844 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.002022957331030844 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.04498894793099842 * Z(2) + 0.025791742201156664 * (Z(0) @ Z(2)) + 0.0021181739685779463 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.0021181739685779463 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.006238714611186875 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.006238714611186875 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.0384102027433099 * Z(4) + 0.03299377398000801 * (Z(0) @ Z(4)) + 0.032312626825255625 * Z(6) + 0.038663422764243525 * (Z(0) @ Z(6)) + 0.05063641233976043 * Z(1) + 0.07316430147426889 * (Z(0) @ Z(1)) + 0.008899911824867397 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.008899911824867397 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.0451266770146801 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.0451266770146801 * (Y(0) @ Y(1) @ X(2) @ X(3))

/opt/miniconda3/envs/penny/lib/python3.12/site-packages/autoray/autoray.py:1127: ComplexWarning: Casting complex values to real discards the imaginary part
  return x.astype(dtype, **kwargs)


The expectation value of a⁺(0) a(6) is 2.3033058381538586e-18
The current excitation operator is a⁺(1) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(5) is 4.629126690650076e-08
The current excitation operator is a⁺(1) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(7) is 2.6051562322268105e-18
The current excitation operator is a⁺(2) a(4)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(4) is 6.6066951348615555e-18
The current excitation operator is a⁺(2) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(6) is 5.849513393108019e-08
The current excitation operator is a⁺(3) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(3) a(5) is 1.8555083756699496e-18
The current excitation operator is a⁺(3) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(3) a(7) is 5.84951339172024e-08
The current excitation operator is a⁺(5) a⁺(4) a(1) a(0)
Pri